In [1]:
# Pkg.add("LowRankModels")
# Pkg.update()
# Pkg.add("SCS")

In [2]:
# Pkg.add("Gadfly")
using Convex
using DataFrames
using PyPlot
using SCS

In [3]:
include("proxgrad.jl")

proxgrad_const (generic function with 1 method)

In [4]:
PERCENT_TEST = 0.2
PERCENT_VALIDATE = 0.25
;

In [5]:
# Generate a split data with a given percent from 0-1
function splitData(listings, percent)
    n = size(listings)[1]
    # Code borrowed from http://blog.yhat.com/posts/julia-neural-networks.html
    is_split = shuffle(1:n .> floor(n * percent))
    listings_train, listings_test = listings[is_split, :], listings[!is_split, :]
    return listings_train, listings_test
end

splitData (generic function with 1 method)

In [6]:
# load data
listings = readtable("listings_cleaned.csv")
listings[:offset] = 1

# Generate train/test split
listings_train, listings_test = splitData(listings, PERCENT_TEST)
listings = listings_train;

In [7]:
# Generate train/validate split
listings_train, listings_validate = splitData(listings, PERCENT_VALIDATE)
listings = listings_train;

In [8]:
# Various loss and error functions

function lasso(X,y; λ=1)
    @show d = size(X,2)
    @show w = Variable(d)
    @show p = minimize(sumsquares(X*w - y) + λ*norm(w,1))
    solve!(p)
    return w.value
end

function ridge_regression(X,y; λ=1)
    d = size(X,2)
    w = Variable(d)
    p = minimize(sumsquares(X*w - y) + λ*sumsquares(w))
    solve!(p)
    return w.value
end

function nnls(X,y)
    d = size(X,2)
    w = Variable(d)
    p = minimize(sumsquares(X*w - y), w>=0)
    solve!(p)
    return w.value
end

function ols(x, y)
    return x\y
end

function RMSE(w, x, y)
    n = length(y)
    f(x_i) = vecdot(w,x_i)
    total_error = 0
    for i = 1:size(x,1)
        actual = y[i]
        predicted = f(x[i,:])
        total_error += (actual - predicted)^2
    end
    return sqrt(total_error / n)
end

function squared_error(w, x, y)
    f(x_i) = vecdot(w,x_i)
    total_error = 0
    for i = 1:size(x,1)
        actual = y[i]
        predicted = f(x[i,:])
        total_error += (actual - predicted)^2
    end
    return total_error
end

function abs_error(w, x, y)
    f(x_i) = vecdot(w,x_i)
    total_error = 0
    for i = 1:size(x,1)
        actual = y[i]
        predicted = f(x[i,:])
        total_error += abs(actual - predicted)
    end
    return total_error
end

abs_error (generic function with 1 method)

In [9]:
# Takes in training and validation vectors, and returns the error for a specific method (as well as the w)
function calculateError(x_train, y_train, x_validate, y_validate, lossFunction, regFunction)
    n = length(y_train)
    w = proxgrad((1./n)*lossFunction, regFunction, x_train, y_train, maxiters=200)
    
    err = RMSE(w, x_validate, y_validate)
    return err, w
end

function calculateErrorOLS(x_train, y_train, x_validate, y_validate)
    w = ols(x_train, y_train)
    err = RMSE(w, x_validate, y_validate)
    return err, w
end

calculateErrorOLS (generic function with 1 method)

In [10]:
# Cross validate to produce best w
k = 5

function crossValidate(columnList, lossFunction, regFunction)
    minW = nothing
    minErr = typemax(Int32)
    for i in 1:k
        listings_training, listings_validate = splitData(listings, PERCENT_VALIDATE) #(listings_train)
        
        x = convert(Array{Float64}, listings_training[:, columnList])
        y = convert(Array{Float64}, listings_training[:, [:price]])
        y = y[:,1] #???
        
        x_validate = convert(Array{Float64}, listings_validate[:, columnList])
        y_validate = convert(Array{Float64}, listings_validate[:, [:price]])
        y_validate = y_validate[:,1] #????
        
        err, w = calculateError(x, y, x_validate, y_validate, lossFunction, regFunction)
        if (err < minErr)
            minW = w
            minErr = err
        end
        
        #TODO: Use average not min? (in sample error)
    end
    return minW, minErr
end

crossValidate (generic function with 1 method)

In [11]:
# Choose different sets of columns that we care about

columns_1 = [:accommodates, :beds, :amenities, :review_scores_rating, :offset]
columns_2 = [:accommodates, :beds, :offset]
columns_3 = [:accommodates, :beds, :room_type_entire_home_apt, :offset]
columns_4 = [:accommodates, :beds, :room_type_entire_home_apt, :reviews_per_month, :offset]
columns_5 = [:accommodates, :beds, :bed_type_real_bed, :room_type_entire_home_apt, :reviews_per_month, :property_type_apartment, :offset]
columns_6 = [:accommodates, :beds, :bed_type_real_bed, :room_type_entire_home_apt, :room_type_private_room
, :room_type_shared_room, :reviews_per_month, :property_type_apartment, :offset]
;

In [12]:

#huber regression
w_huber1, err_huber1 = crossValidate(columns_1, HuberLoss(), ZeroReg())
w_huber2, err_huber2 = crossValidate(columns_2, HuberLoss(), ZeroReg())
w_huber3, err_huber3 = crossValidate(columns_3, HuberLoss(), ZeroReg())
w_huber4, err_huber4 = crossValidate(columns_4, HuberLoss(), ZeroReg())
w_huber5, err_huber5 = crossValidate(columns_5, HuberLoss(), ZeroReg())
@show err_huber1
@show err_huber2
@show err_huber3
@show err_huber4
@show err_huber5

# ordinary least squares regression
lambda = 1
w_quad1, err_quad1 = crossValidate(columns_1, QuadLoss(), ZeroReg())
w_quad2, err_quad2 = crossValidate(columns_2, QuadLoss(), ZeroReg())
w_quad3, err_quad3 = crossValidate(columns_3, QuadLoss(), ZeroReg())
w_quad4, err_quad4 = crossValidate(columns_4, QuadLoss(), ZeroReg())
w_quad5, err_quad5 = crossValidate(columns_5, QuadLoss(), ZeroReg())
w_quad5_1, err_quad5_1 = crossValidate(columns_5, QuadLoss(), OneReg(lambda))
w_quad5_2, err_quad5_2 = crossValidate(columns_5, QuadLoss(), QuadReg(lambda))
w_quad6, err_quad6 = crossValidate(columns_6, QuadLoss(), ZeroReg())
@show err_quad1
@show err_quad2
@show err_quad3
@show err_quad4
@show err_quad5
@show err_quad5_1
@show err_quad5_2
@show err_quad6

# l1 loss regression
w_l11, err_l11 = crossValidate(columns_1, L1Loss(), ZeroReg())
w_l12, err_l12 = crossValidate(columns_2, L1Loss(), ZeroReg())
w_l13, err_l13 = crossValidate(columns_3, L1Loss(), ZeroReg())
w_l14, err_l14 = crossValidate(columns_4, L1Loss(), ZeroReg())
w_l15, err_l15 = crossValidate(columns_5, L1Loss(), ZeroReg())
@show err_l11
@show err_l12
@show err_l13
@show err_l14
@show err_l15

# quantile regression
w_quantile1, err_quantile1 = crossValidate(columns_5, QuantileLoss(quantile=.4), ZeroReg())
@show err_quantile1

err_huber1 = 282.2436754753142
err_huber2 = 306.7544184914401
err_huber3 = 362.386573267509
err_huber4 = 272.3301890334482
err_huber5 = 305.9722946750409
err_quad1 = 302.92041884184084
err_quad2 = 287.35099089235587
err_quad3 = 266.33099696331607
err_quad4 = 204.57579006675235
err_quad5 = 247.1767223633175
err_quad5_1 = 284.8034250368456
err_quad5_2 = 269.0582951717673
err_quad6 = 280.4831176975978
err_l11 = 295.2282127089493
err_l12 = 303.8164556042745
err_l13 = 313.38448743765167
err_l14 = 344.5732982610158
err_l15 = 282.3862288581897
err_quantile1 = 289.6266000056086


289.6266000056086

In [ ]:
# ordinary least squares

@show calculateErrorOLS(x1, y, x1_test, y_test)
@show calculateErrorOLS(x2, y, x2_test, y_test)
@show calculateErrorOLS(x3, y, x3_test, y_test)
@show calculateErrorOLS(x4, y, x4_test, y_test)
@show calculateErrorOLS(x5, y, x5_test, y_test)

In [ ]:
w_ridge1 = ridge_regression(x1, y)
@show err_ridge1 = RMSE(w_ridge1, x1_test, y_test)


In [ ]:
w_lasso1 = lasso(x1, y)
@show err_lasso1 = RMSE(w_lasso1, x1_test, y_test)


In [13]:
# Test our w's

function calculateTestError(listings_test, columnList, w)
    x_test = convert(Array{Float64}, listings_test[:, columnList])
    y_test = convert(Array{Float64}, listings_test[:, [:price]])
    err = RMSE(w, x_test, y_test)
    return err
end

@show calculateTestError(listings_test, columns_1, w_huber1)
@show calculateTestError(listings_test, columns_2, w_huber2)
@show calculateTestError(listings_test, columns_3, w_huber3)
@show calculateTestError(listings_test, columns_4, w_huber4)
@show calculateTestError(listings_test, columns_5, w_huber5)

@show calculateTestError(listings_test, columns_1, w_quad1)
@show calculateTestError(listings_test, columns_2, w_quad2)
@show calculateTestError(listings_test, columns_3, w_quad3)
@show calculateTestError(listings_test, columns_4, w_quad4)
@show calculateTestError(listings_test, columns_5, w_quad5)
@show calculateTestError(listings_test, columns_5, w_quad5_1)
@show calculateTestError(listings_test, columns_5, w_quad5_2)
@show calculateTestError(listings_test, columns_6, w_quad6)

@show calculateTestError(listings_test, columns_1, w_l11)
@show calculateTestError(listings_test, columns_2, w_l12)
@show calculateTestError(listings_test, columns_3, w_l13)
@show calculateTestError(listings_test, columns_4, w_l14)
@show calculateTestError(listings_test, columns_5, w_l15)

@show calculateTestError(listings_test, columns_5, w_quantile1)

calculateTestError(listings_test,columns_1,w_huber1) = 344.3418152947554
calculateTestError(listings_test,columns_2,w_huber2) = 317.90502218973086
calculateTestError(listings_test,columns_3,w_huber3) = 319.1040029730845
calculateTestError(listings_test,columns_4,w_huber4) = 317.7660734117339
calculateTestError(listings_test,columns_5,w_huber5) = 317.9455430860012
calculateTestError(listings_test,columns_1,w_quad1) = 319.9414041618917
calculateTestError(listings_test,columns_2,w_quad2) = 308.1537329454
calculateTestError(listings_test,columns_3,w_quad3) = 308.6419036652755
calculateTestError(listings_test,columns_4,w_quad4) = 306.4425450201415
calculateTestError(listings_test,columns_5,w_quad5) = 304.2096691352894
calculateTestError(listings_test,columns_5,w_quad5_1) = 304.6400752834774
calculateTestError(listings_test,columns_5,w_quad5_2) = 304.66948621990684
calculateTestError(listings_test,columns_6,w_quad6) = 304.11176851875587
calculateTestError(listings_test,columns_1,w_l11) = 344

323.8158636357374

In [ ]:
plt[:hist](w_ridge1)

In [ ]:
plt[:hist](w_lasso1, bins=50)